In [ ]:
# Goal: manipulate qiskit programs

In [1]:
import pandas as pd
import sys
sys.path.append("../lib")
import seaborn as sns
import matplotlib.pyplot as plt
from utils import iterate_over
from utils import load_config_and_check
from inspector import Inspector
from tqdm import tqdm
import sqlite3 as sl
import os

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Useful additional packages
import matplotlib.pyplot as plt
import numpy as np
from math import pi

In [4]:
import qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer

In [5]:
from metamorph import get_sections
from metamorph import get_registers_used

In [6]:
QISKIT_PROGRAM_PATH = "stub_files/sample_qiskit_circuit.py"
QISKIT_PROGRAM_CONTENT = open(QISKIT_PROGRAM_PATH, 'r').read()
print(QISKIT_PROGRAM_CONTENT)

# SECTION
# NAME: PROLOGUE
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

# SECTION
# NAME: CIRCUIT
qr_0 = QuantumRegister(2)
c_0 = QuantumCircuit(qr_0)

c_0.h(qr_0[0])
c_0.crz(1, qr_0[0], qr_0[1])
c_0.barrier()
c_0.id(qr_0[1])
c_0.u(1, 2, -2, qr_0[0])
c_0.u(2, 3, -2, qr_0[0])
c_0.u(3, 4, -2, qr_0[0])


# SECTION
# NAME: OPTIMIZATION

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Optimize1qGates

passmanager = PassManager()
passmanager.append(Optimize1qGates())


# SECTION
# NAME: EXECUTION
from qiskit import Aer, transpile, execute

shots = 8192

c_0.measure_all()

backend = Aer.get_backend('qasm_simulator')
job = execute(c_0, backend=backend, shots=shots)
job_result = job.result()
count = job_result.get_counts(c_0)
print(count)

RESULT = count


# Convert to a gate and stick it into an arbitrary place in the bigger circuit
#sub_inst = sub_circ.to_instruction()

#qr = QuantumRegister(3, 'q')
#circ = QuantumCircuit(qr)
#circ.h(qr

In [7]:
sections = get_sections(QISKIT_PROGRAM_CONTENT)
circuit_code = sections["CIRCUIT"]

In [8]:
import ast
import astpretty

In [9]:
tree = ast.parse(circuit_code)

In [10]:
astpretty.pprint(tree)

Module(
    body=[
        Assign(
            lineno=2,
            col_offset=0,
            end_lineno=2,
            end_col_offset=25,
            targets=[Name(lineno=2, col_offset=0, end_lineno=2, end_col_offset=4, id='qr_0', ctx=Store())],
            value=Call(
                lineno=2,
                col_offset=7,
                end_lineno=2,
                end_col_offset=25,
                func=Name(lineno=2, col_offset=7, end_lineno=2, end_col_offset=22, id='QuantumRegister', ctx=Load()),
                args=[Constant(lineno=2, col_offset=23, end_lineno=2, end_col_offset=24, value=2, kind=None)],
                keywords=[],
            ),
            type_comment=None,
        ),
        Assign(
            lineno=3,
            col_offset=0,
            end_lineno=3,
            end_col_offset=26,
            targets=[Name(lineno=3, col_offset=0, end_lineno=3, end_col_offset=3, id='c_0', ctx=Store())],
            value=Call(
                lineno=3,
              

## Generate a Circuit from a Qiskit algorithm

In [50]:
from qiskit.algorithms import AmplitudeEstimation
from qiskit.algorithms import EstimationProblem

In [54]:
class BernoulliStateIn(QuantumCircuit):
    """A circuit preparing sqrt(1 - p)|0> + sqrt(p)|1>."""

    def __init__(self, probability):
        super().__init__(1)
        angle = 2 * np.arcsin(np.sqrt(probability))
        self.ry(angle, 0)

prob = 0.8
a_op = BernoulliStateIn(prob)
problem = EstimationProblem(a_op, objective_qubits=0)
qc = AmplitudeEstimation(4).construct_circuit(estimation_problem=problem)

In [56]:
qc.draw()

┌──────┐
eval_0: ──────────────┤0     ├
                      │      │
eval_1: ──────────────┤1     ├
                      │      │
eval_2: ──────────────┤2 QPE ├
                      │      │
eval_3: ──────────────┤3     ├
        ┌────────────┐│      │
     q: ┤ Ry(2.2143) ├┤4     ├
        └────────────┘└──────┘

In [69]:
qasm_conversion = qc.qasm()
qasm_conversion = qasm_conversion.replace(";", ";\n")
print(qasm_conversion)

OPENQASM 2.0;

include "qelib1.inc";

gate gate_IQFT q0,q1,q2,q3 { h q0;
 cp(-pi/2) q1,q0;
 h q1;
 cp(-pi/4) q2,q0;
 cp(-pi/2) q2,q1;
 h q2;
 cp(-pi/8) q3,q0;
 cp(-pi/4) q3,q1;
 cp(-pi/2) q3,q2;
 h q3;
 }
gate cQ__8 q0,q1 { cu(pi/2,0,pi,0) q0,q1;
 cx q0,q1;
 cu(pi/2,0,pi,0) q0,q1;
 p(0) q0;
 p(0) q1;
 cx q0,q1;
 u(1.1071487,0,0) q1;
 cx q0,q1;
 u(-1.1071487,0,0) q1;
 cx q0,q1;
 h q1;
 cx q0,q1;
 h q1;
 cx q0,q1;
 p(0) q0;
 p(0) q1;
 cx q0,q1;
 u(-1.1071487,0,0) q1;
 cx q0,q1;
 u(1.1071487,0,0) q1;
 cu(pi/2,0,pi,0) q0,q1;
 cx q0,q1;
 cu(pi/2,0,pi,0) q0,q1;
 p(0) q0;
 p(0) q1;
 cx q0,q1;
 u(1.1071487,0,0) q1;
 cx q0,q1;
 u(-1.1071487,0,0) q1;
 cx q0,q1;
 h q1;
 cx q0,q1;
 h q1;
 cx q0,q1;
 p(0) q0;
 p(0) q1;
 cx q0,q1;
 u(-1.1071487,0,0) q1;
 cx q0,q1;
 u(1.1071487,0,0) q1;
 cu(pi/2,0,pi,0) q0,q1;
 cx q0,q1;
 cu(pi/2,0,pi,0) q0,q1;
 p(0) q0;
 p(0) q1;
 cx q0,q1;
 u(1.1071487,0,0) q1;
 cx q0,q1;
 u(-1.1071487,0,0) q1;
 cx q0,q1;
 h q1;
 cx q0,q1;
 h q1;
 cx q0,q1;
 p(0) q0;
 p(0) q1;
 cx 

In [58]:
qc.decompose().draw()

┌───┐                                      ┌───────┐
eval_0: ──────┤ H ├─────────■────────────────────────────┤3      ├
              ├───┤         │                            │       │
eval_1: ──────┤ H ├─────────┼───────■────────────────────┤2      ├
              ├───┤         │       │                    │  IQFT │
eval_2: ──────┤ H ├─────────┼───────┼───────■────────────┤1      ├
              ├───┤         │       │       │            │       │
eval_3: ──────┤ H ├─────────┼───────┼───────┼───────■────┤0      ├
        ┌─────┴───┴─────┐┌──┴───┐┌──┴───┐┌──┴───┐┌──┴───┐└───────┘
     q: ┤ R(2.2143,π/2) ├┤ Q**1 ├┤ Q**2 ├┤ Q**4 ├┤ Q**8 ├─────────
        └───────────────┘└──────┘└──────┘└──────┘└──────┘

In [64]:
qc.decompose().decompose().draw()

┌─────────┐                                         ┌──────┐     »
eval_0: ───┤ U2(0,π) ├───────────■──────────■─────────■────────┤ P(0) ├──■──»
           ├─────────┤           │          │         │        └──────┘  │  »
eval_1: ───┤ U2(0,π) ├───────────┼──────────┼─────────┼──────────────────┼──»
           ├─────────┤           │          │         │                  │  »
eval_2: ───┤ U2(0,π) ├───────────┼──────────┼─────────┼──────────────────┼──»
           ├─────────┤           │          │         │                  │  »
eval_3: ───┤ U2(0,π) ├───────────┼──────────┼─────────┼──────────────────┼──»
        ┌──┴─────────┴───┐┌──────┴───────┐┌─┴─┐┌──────┴───────┐┌──────┐┌─┴─┐»
     q: ┤ U3(2.2143,0,0) ├┤ U(π/2,0,π,0) ├┤ X ├┤ U(π/2,0,π,0) ├┤ P(0) ├┤ X ├»
        └────────────────┘└──────────────┘└───┘└──────────────┘└──────┘└───┘»
«                                                                         »
«eval_0: ───────────────────■──────────────────────■─────────■─────────■──»
«                           │                      │         │         │  »
«eval_1: ───────────────────┼──────────────────────┼─────────┼─────────┼──»
«                           │                      │         │         │  »
«eval_2: ───────────────────┼──────────────────────┼─────────┼─────────┼──»
«                           │                      │         │         │  »
«eval_3: ───────────────────┼──────────────────────┼─────────┼─────────┼──»
«        ┌───────────────┐┌─┴─┐┌────────────────┐┌─┴─┐┌───┐┌─┴─┐┌───┐┌─┴─┐»
«     q: ┤ U(1.1071,0,0) ├┤ X ├┤ U(-1.1071,0,0) ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├»
«        └───────────────┘└───┘└────────────────┘└───┘└───┘└───┘└───┘└───┘»
«        ┌──────┐                                 ┌──────┐                    »
«eval_0: ┤ P(0) ├──■──────────────────────■───────┤ P(π) ├────────────────────»
«        └──────┘  │                      │       └──────┘                    »
«eval_1: ──────────┼──────────────────────┼──────────────────────────■────────»
«                  │                      │                          │        »
«eval_2: ──────────┼──────────────────────┼──────────────────────────┼────────»
«                  │                      │                          │        »
«eval_3: ──────────┼──────────────────────┼──────────────────────────┼────────»
«        ┌──────┐┌─┴─┐┌────────────────┐┌─┴─┐┌───────────────┐┌──────┴───────┐»
«     q: ┤ P(0) ├┤ X ├┤ U(-1.1071,0,0) ├┤ X ├┤ U(1.1071,0,0) ├┤ U(π/2,0,π,0) ├»
«        └──────┘└───┘└────────────────┘└───┘└───────────────┘└──────────────┘»
«                                                                »
«eval_0: ────────────────────────────────────────────────────────»
«                             ┌──────┐                           »
«eval_1: ──■─────────■────────┤ P(0) ├──■─────────────────────■──»
«          │         │        └──────┘  │                     │  »
«eval_2: ──┼─────────┼──────────────────┼─────────────────────┼──»
«          │         │                  │                     │  »
«eval_3: ──┼─────────┼──────────────────┼─────────────────────┼──»
«        ┌─┴─┐┌──────┴───────┐┌──────┐┌─┴─┐┌───────────────┐┌─┴─┐»
«     q: ┤ X ├┤ U(π/2,0,π,0) ├┤ P(0) ├┤ X ├┤ U(1.1071,0,0) ├┤ X ├»
«        └───┘└──────────────┘└──────┘└───┘└───────────────┘└───┘»
«                                                                »
«eval_0: ────────────────────────────────────────────────────────»
«                                                   ┌──────┐     »
«eval_1: ────────────────────■─────────■─────────■──┤ P(0) ├──■──»
«                            │         │         │  └──────┘  │  »
«eval_2: ────────────────────┼─────────┼─────────┼────────────┼──»
«                            │         │         │            │  »
«eval_3: ────────────────────┼─────────┼─────────┼────────────┼──»
«        ┌────────────────┐┌─┴─┐┌───┐┌─┴─┐┌───┐┌─┴─┐┌──────┐┌─┴─┐»
«     q: ┤ U(-1.1071,0,0) ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ P(0) ├┤ X ├»
«        └────────────────┘└──

In [70]:
qc_decomposed = qc.decompose().decompose().decompose().decompose().decompose().decompose().decompose()
qasm_conversion = qc_decomposed.qasm()
qasm_conversion = qasm_conversion.replace(";", ";\n")
print(qasm_conversion)

OPENQASM 2.0;

include "qelib1.inc";

qreg eval[4];

qreg q[1];

u(pi/2,0,pi) eval[0];

u(0,0,0) eval[0];

u(0,0,pi/2) eval[0];

u(pi/2,0,pi) eval[1];

u(0,0,0) eval[1];

u(0,0,pi/2) eval[1];

u(pi/2,0,pi) eval[2];

u(0,0,0) eval[2];

u(0,0,pi/2) eval[2];

u(pi/2,0,pi) eval[3];

u(0,0,0) eval[3];

u(0,0,pi/2) eval[3];

u(2.2142974,0,0) q[0];

u(0,0,pi/2) q[0];

cx eval[0],q[0];

u(-pi/4,0,-pi/2) q[0];

cx eval[0],q[0];

u(pi/4,0,0) q[0];

cx eval[0],q[0];

u(0,0,pi/2) q[0];

u(0,0,0) eval[0];

u(0,0,pi/2) eval[0];

cx eval[0],q[0];

u(-pi/4,0,-pi/2) q[0];

cx eval[0],q[0];

u(pi/4,0,0) q[0];

u(0,0,0) q[0];

u(0,0,0) eval[0];

cx eval[0],q[0];

u(1.1071487,0,0) q[0];

cx eval[0],q[0];

u(-1.1071487,0,0) q[0];

cx eval[0],q[0];

u(pi/2,0,pi) q[0];

cx eval[0],q[0];

u(pi/2,0,pi) q[0];

cx eval[0],q[0];

u(0,0,0) q[0];

u(0,0,0) eval[0];

cx eval[0],q[0];

u(-1.1071487,0,0) q[0];

cx eval[0],q[0];

u(1.1071487,0,0) q[0];

u(0,0,pi/2) q[0];

u(0,0,pi) eval[0];

u(0,0,-pi/16) eval[0];

cx 

## Read Registers from AST

In [11]:
registers = get_registers_used(circ_definition=circuit_code)
registers

[{'name': 'qr_0', 'type': 'QuantumRegister', 'size': 2}]

In [94]:
GLOBALS = {"RESULT": 0}

exec(QISKIT_PROGRAM_CONTENT, GLOBALS)

{'00': 3058, '01': 5134}


In [96]:
GLOBALS["RESULT"]

{'00': 3058, '01': 5134}

## Auto Generation of Qiskit Code - Circuit

In [36]:
from generation_strategy_python import QiskitFuzzer

In [112]:
config = load_config_and_check("../config/qmt_v00.yaml")

In [90]:
fuzzer = QiskitFuzzer()

In [46]:
source_code, metadata = fuzzer.generate_via_atomic_ops(
    gate_set=config["generation_strategy"]["gate_set"],
    n_qubits=5,
    n_ops=10
)

In [53]:
[b.name for b in backends]

[<bound method AerSimulator.name of AerSimulator('aer_simulator')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_statevector')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_statevector_gpu')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_density_matrix')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_density_matrix_gpu')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_stabilizer')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_matrix_product_state')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_extended_stabilizer')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_unitary')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_unitary_gpu')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_superop')>,
 <bound method BackendV1.name of QasmSimulator('qasm_simulator')>,
 <bound method BackendV1.name of Statevecto

In [104]:
long_str = """[<bound method AerSimulator.name of AerSimulator('aer_simulator')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_statevector')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_statevector_gpu')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_density_matrix')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_density_matrix_gpu')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_stabilizer')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_matrix_product_state')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_extended_stabilizer')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_unitary')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_unitary_gpu')>,
 <bound method AerSimulator.name of AerSimulator('aer_simulator_superop')>,
 <bound method BackendV1.name of QasmSimulator('qasm_simulator')>,
 <bound method BackendV1.name of StatevectorSimulator('statevector_simulator')>,
 <bound method BackendV1.name of UnitarySimulator('unitary_simulator')>,
 <bound method BackendV1.name of PulseSimulator('pulse_simulator')>]"""

for line in long_str.split("\n"):
    print("- " + line.split("('")[1].replace("')>", "").replace(",", ""))

- aer_simulator
- aer_simulator_statevector
- aer_simulator_statevector_gpu
- aer_simulator_density_matrix
- aer_simulator_density_matrix_gpu
- aer_simulator_stabilizer
- aer_simulator_matrix_product_state
- aer_simulator_extended_stabilizer
- aer_simulator_unitary
- aer_simulator_unitary_gpu
- aer_simulator_superop
- qasm_simulator
- statevector_simulator
- unitary_simulator
- pulse_simulator]


In [106]:
type(np.random.choice([1,2,3]))

numpy.int64

In [115]:
generated_py_file, metadata = fuzzer.generate_file(
    gate_set=config["generation_strategy"]["gate_set"],
    n_qubits=5,
    n_ops=10,
    optimizations=["CommutationAnalysis", "CommutativeCancellation", "CXCancellation"],
    backend="qasm_simulator",
    shots=8192)
print(generated_py_file)


# SECTION
# NAME: PROLOGUE

import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library.standard_gates import *

# SECTION
# NAME: CIRCUIT

qr_qr = QuantumRegister(5, name='qr_qr')
cr_qr = ClassicalRegister(5, name='cr_qr')
qc = QuantumCircuit(qr_qr, cr_qr, name='qc')
qc.append(CSwapGate(), qargs=[qr_qr[3], qr_qr[0], qr_qr[1]], cargs=[])
qc.append(XGate(), qargs=[qr_qr[0]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[2], qr_qr[3]], cargs=[])
qc.append(CHGate(), qargs=[qr_qr[4], qr_qr[0]], cargs=[])
qc.append(RZZGate(3.381309563488947), qargs=[qr_qr[4], qr_qr[1]], cargs=[])
qc.append(U2Gate(3.4690789366725516,6.089828077752466), qargs=[qr_qr[2]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[2], qr_qr[1]], cargs=[])
qc.append(RZXGate(4.008916676545142), qargs=[qr_qr[2], qr_qr[1]], cargs=[])
qc.append(RC3XGate(), qargs=[qr_qr[2], qr_qr[0], qr_qr[1], qr_qr[3]], cargs=[])
qc.append(RCCXGate(), qargs=[qr_qr[1], qr_qr[0], qr_qr[2]], cargs=

In [113]:
from metamorph import change_backend

In [142]:
modified_code = change_backend(source_code=generated_py_file, available_backends=config["generation_strategy"]["backends"])

#astpretty.pprint(modified_code)

Found: qasm_simulator


In [143]:
print(modified_code)

# SECTION
# NAME: PROLOGUE

import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library.standard_gates import *
# SECTION
# NAME: CIRCUIT

qr_qr = QuantumRegister(5, name='qr_qr')
cr_qr = ClassicalRegister(5, name='cr_qr')
qc = QuantumCircuit(qr_qr, cr_qr, name='qc')
qc.append(CSwapGate(), qargs=[qr_qr[3], qr_qr[0], qr_qr[1]], cargs=[])
qc.append(XGate(), qargs=[qr_qr[0]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[2], qr_qr[3]], cargs=[])
qc.append(CHGate(), qargs=[qr_qr[4], qr_qr[0]], cargs=[])
qc.append(RZZGate(3.381309563488947), qargs=[qr_qr[4], qr_qr[1]], cargs=[])
qc.append(U2Gate(3.4690789366725516, 6.089828077752466), qargs=[qr_qr[2]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[2], qr_qr[1]], cargs=[])
qc.append(RZXGate(4.008916676545142), qargs=[qr_qr[2], qr_qr[1]], cargs=[])
qc.append(RC3XGate(), qargs=[qr_qr[2], qr_qr[0], qr_qr[1], qr_qr[3]], cargs=[])
qc.append(RCCXGate(), qargs=[qr_qr[1], qr_qr[0], qr_qr[2]], cargs=[

In [137]:
!pip install astor

  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)


In [124]:
print(modified_code)

# SECTION
# NAME: PROLOGUE

import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library.standard_gates import *
# SECTION
# NAME: CIRCUIT

qr_qr = QuantumRegister(5, name='qr_qr')
cr_qr = ClassicalRegister(5, name='cr_qr')
qc = QuantumCircuit(qr_qr, cr_qr, name='qc')
qc.append(CSwapGate(), qargs=[qr_qr[3], qr_qr[0], qr_qr[1]], cargs=[])
qc.append(XGate(), qargs=[qr_qr[0]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[2], qr_qr[3]], cargs=[])
qc.append(CHGate(), qargs=[qr_qr[4], qr_qr[0]], cargs=[])
qc.append(RZZGate(3.381309563488947), qargs=[qr_qr[4], qr_qr[1]], cargs=[])
qc.append(U2Gate(3.4690789366725516, 6.089828077752466), qargs=[qr_qr[2]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[2], qr_qr[1]], cargs=[])
qc.append(RZXGate(4.008916676545142), qargs=[qr_qr[2], qr_qr[1]], cargs=[])
qc.append(RC3XGate(), qargs=[qr_qr[2], qr_qr[0], qr_qr[1], qr_qr[3]], cargs=[])
qc.append(RCCXGate(), qargs=[qr_qr[1], qr_qr[0], qr_qr[2]], cargs=[

In [111]:
astpretty.pprint(ast.parse('Aer.get_backend("qasm_simulator")', mode='eval'), indent=4)

Expression(
    body=Call(
        lineno=1,
        col_offset=0,
        end_lineno=1,
        end_col_offset=33,
        func=Attribute(
            lineno=1,
            col_offset=0,
            end_lineno=1,
            end_col_offset=15,
            value=Name(lineno=1, col_offset=0, end_lineno=1, end_col_offset=3, id='Aer', ctx=Load()),
            attr='get_backend',
            ctx=Load(),
        ),
        args=[Constant(lineno=1, col_offset=16, end_lineno=1, end_col_offset=32, value='qasm_simulator', kind=None)],
        keywords=[],
    ),
)


In [101]:
GLOBALS = {"RESULT": 0}

exec(generated_py_file, GLOBALS)

In [102]:
GLOBALS["RESULT"]

{'11000': 10,
 '00100': 3853,
 '01100': 3609,
 '10000': 5,
 '10100': 178,
 '11100': 195,
 '01000': 154,
 '00000': 188}

In [ ]:
qr_qr = QuantumRegister(5, name='qr_qr')
cr_qr = ClassicalRegister(5, name='cr_qr')
qc = QuantumCircuit(qr_qr, cr_qr, name='qc')
qc.append(CZGate(), qargs=[qr_qr[4], qr_qr[3]], cargs=[])
qc.append(IGate(), qargs=[qr_qr[1]], cargs=[])
qc.append(C3SXGate(), qargs=[qr_qr[3], qr_qr[0], qr_qr[2]], cargs=[])
qc.append(RXGate(2.700932295903693), qargs=[qr_qr[0]], cargs=[])
qc.append(U3Gate(0.09144240096955454,5.910826577430733,1.3545764490066696), qargs=[qr_qr[3]], cargs=[])
qc.append(SwapGate(), qargs=[qr_qr[4], qr_qr[1]], cargs=[])
qc.append(SGate(), qargs=[qr_qr[0]], cargs=[])
qc.append(C3SXGate(), qargs=[qr_qr[3], qr_qr[0], qr_qr[1]], cargs=[])
qc.append(RYYGate(1.3519467402030274), qargs=[qr_qr[1], qr_qr[3]], cargs=[])
qc.append(RCCXGate(), qargs=[qr_qr[0], qr_qr[1], qr_qr[3]], cargs=[])

---------------------------------------------------------------------------
CircuitError                              Traceback (most recent call last)
/tmp/ipykernel_33593/513803224.py in <module>
      1 GLOBALS = {"RESULT": 0}
      2 
----> 3 exec(generated_py_file, GLOBALS)

<string> in <module>

~/.conda/envs/ML4Quantum/lib/python3.8/site-packages/qiskit/circuit/quantumcircuit.py in append(self, instruction, qargs, cargs)
   1228             requester = self._resolve_classical_resource
   1229         instructions = InstructionSet(resource_requester=requester)
-> 1230         for qarg, carg in instruction.broadcast_arguments(expanded_qargs, expanded_cargs):
   1231             instructions.add(appender(instruction, qarg, carg), qarg, carg)
   1232         return instructions

~/.conda/envs/ML4Quantum/lib/python3.8/site-packages/qiskit/circuit/gate.py in broadcast_arguments(self, qargs, cargs)
    198         """
    199         if len(qargs) != self.num_qubits or cargs:
--> 200             raise CircuitError(
    201                 f"The amount of qubit({len(qargs)})/clbit({len(cargs)}) arguments does"
    202                 f" not match the gate expectation ({self.num_qubits})."

CircuitError: 'The amount of qubit(3)/clbit(0) arguments does not match the gate expectation (4).'

In [145]:
# SECTION
# NAME: PROLOGUE

import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library.standard_gates import *

# SECTION
# NAME: CIRCUIT

qr_qr = QuantumRegister(13, name='qr_qr')
cr_qr = ClassicalRegister(13, name='cr_qr')
qc = QuantumCircuit(qr_qr, cr_qr, name='qc')
qc.append(PhaseGate(3.9609333943213456), qargs=[qr_qr[9]], cargs=[])
qc.draw(fold=-1)

qr_qr_0: ─────────────
                       
 qr_qr_1: ─────────────
                       
 qr_qr_2: ─────────────
                       
 qr_qr_3: ─────────────
                       
 qr_qr_4: ─────────────
                       
 qr_qr_5: ─────────────
                       
 qr_qr_6: ─────────────
                       
 qr_qr_7: ─────────────
                       
 qr_qr_8: ─────────────
          ┌───────────┐
 qr_qr_9: ┤ P(3.9609) ├
          └───────────┘
qr_qr_10: ─────────────
                       
qr_qr_11: ─────────────
                       
qr_qr_12: ─────────────
                       
cr_qr: 13/═════════════

In [ ]:
# SECTION
# NAME: OPTIMIZATION

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import *
passmanager = PassManager()
passmanager.append(RemoveDiagonalGatesBeforeMeasure())
passmanager.append(OptimizeSwapBeforeMeasure())
passmanager.append(Collect2qBlocks())
qc = passmanager.run(qc)

In [146]:
# SECTION
# NAME: MEASUREMENT

qc.measure(qr_qr, cr_qr)
qc.draw(fold=-1)

┌─┐                                 
 qr_qr_0: ─────────────┤M├─────────────────────────────────
                       └╥┘┌─┐                              
 qr_qr_1: ──────────────╫─┤M├──────────────────────────────
                        ║ └╥┘┌─┐                           
 qr_qr_2: ──────────────╫──╫─┤M├───────────────────────────
                        ║  ║ └╥┘┌─┐                        
 qr_qr_3: ──────────────╫──╫──╫─┤M├────────────────────────
                        ║  ║  ║ └╥┘┌─┐                     
 qr_qr_4: ──────────────╫──╫──╫──╫─┤M├─────────────────────
                        ║  ║  ║  ║ └╥┘┌─┐                  
 qr_qr_5: ──────────────╫──╫──╫──╫──╫─┤M├──────────────────
                        ║  ║  ║  ║  ║ └╥┘┌─┐               
 qr_qr_6: ──────────────╫──╫──╫──╫──╫──╫─┤M├───────────────
                        ║  ║  ║  ║  ║  ║ └╥┘┌─┐            
 qr_qr_7: ──────────────╫──╫──╫──╫──╫──╫──╫─┤M├────────────
                        ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐         
 qr_qr_8: ──────────────╫──╫──╫──╫──╫──╫──╫──╫─┤M├─────────
          ┌───────────┐ ║  ║  ║  ║  ║  ║  ║  ║ └╥┘      ┌─┐
 qr_qr_9: ┤ P(3.9609) ├─╫──╫──╫──╫──╫──╫──╫──╫──╫───────┤M├
          └────┬─┬────┘ ║  ║  ║  ║  ║  ║  ║  ║  ║       └╥┘
qr_qr_10: ─────┤M├──────╫──╫──╫──╫──╫──╫──╫──╫──╫────────╫─
               └╥┘      ║  ║  ║  ║  ║  ║  ║  ║  ║ ┌─┐    ║ 
qr_qr_11: ──────╫───────╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├────╫─
                ║       ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐ ║ 
qr_qr_12: ──────╫───────╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├─╫─
                ║       ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘ ║ 
cr_qr: 13/══════╩═══════╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
                10      0  1  2  3  4  5  6  7  8  11 12 9

In [147]:
# SECTION
# NAME: EXECUTION

from qiskit import Aer, transpile, execute
backend_0e3b482c4dd240f5bb9a897a581724c3 = Aer.get_backend('aer_simulator_statevector_gpu')
RESULT = execute(qc, backend=backend_0e3b482c4dd240f5bb9a897a581724c3, shots=15676).result().get_counts(qc)

In [148]:
RESULT

{'0000000000000': 15676}

In [150]:
# SECTION
# NAME: EXECUTION
from qiskit import Aer, transpile, execute
backend_0e3b482c4dd240f5bb9a897a581724c3 = Aer.get_backend('aer_simulator_superop')
RESULT = execute(qc, backend=backend_0e3b482c4dd240f5bb9a897a581724c3,
    shots=15676).result().get_counts(qc)

Simulation failed and returned the following error message:
ERROR:  [Experiment 0] Circuit qc contains invalid instructions {"instructions": {measure}} for "superop" method.


QiskitError: 'Data for experiment "qc" could not be found.'

In [158]:

# SECTION
# NAME: PROLOGUE

import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library.standard_gates import *

# SECTION
# NAME: CIRCUIT

qr_qr = QuantumRegister(10, name='qr_qr')
cr_qr = ClassicalRegister(10, name='cr_qr')
qc = QuantumCircuit(qr_qr, cr_qr, name='qc')
qc.append(CZGate(), qargs=[qr_qr[4], qr_qr[6]], cargs=[])

# SECTION
# NAME: OPTIMIZATION

# SECTION
# NAME: MEASUREMENT

qc.measure(qr_qr, cr_qr)

# SECTION
# NAME: EXECUTION

from qiskit import Aer, transpile, execute
backend_39f225015fb547da844865a411b8f67a = Aer.get_backend('aer_simulator_unitary')
job_result = execute(qc, backend=backend_39f225015fb547da844865a411b8f67a).result().get_unitary()
print(job_result)
#RESULT = job_result..get_unitary(qc)

Simulation failed and returned the following error message:
ERROR:  [Experiment 0] Circuit qc contains invalid instructions {"instructions": {measure}} for "unitary" method.


QiskitError: 'ERROR:  [Experiment 0] Circuit qc contains invalid instructions {"instructions": {measure}} for "unitary" method. ,  ERROR: Circuit qc contains invalid instructions {"instructions": {measure}} for "unitary" method.'